##### En este notebook se automatiza la extracción de datos del DW, segmentación por cantidad de puntos disponibles y la exportación a archivos csv.

Importante: 
- Líneas de código para conectarse directamente al DW.
- Líneas de código para exportar archivos csv y dividirlos si son muy pesados.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.axes as ax
import pyodbc
import math
import os
pd.set_option('display.max_columns', None)

In [ ]:
DB = {'servername': '+++',
      'database': '+++',
      'usuario' :'+++',
      'contraseña':'+++',
      'Encrypt':'yes',
      'TrustServerCertificate':'no',
      'Connection Timeout':'30'}

# create the connection
conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] +\
                      ';UID=' + DB['usuario'] + ';PWD='+ DB['contraseña'] + ';Encrypt='+ DB['Encrypt']+';TrustServerCertificate=' + \
                      DB['TrustServerCertificate']+ ';Connection Timeout=' + DB['Connection Timeout'])

# query db
sql = """

SELECT cpl.CustomerId, sum(cpl.Points) 'Puntos', c.FirstName, c.Email
FROM dbo.CustomerPointsLog cpl
INNER JOIN dbo.Customer c ON c.CustomerId = cpl.CustomerId
GROUP BY cpl.CustomerId, c.FirstName, c.Email
HAVING sum(cpl.Points) > 1799
"""
df = pd.read_sql(sql, conn)
df.head()

In [ ]:
#Como ya me pasan los datos que necesito para df, importo directamente.

#df = pd.read_csv(r'C:/Users/pnovero/Documents/Extractos DW/puntos al 07-12.csv')
#df.drop(index=df.index[0], axis=0, inplace=True)
#df["Puntos"] = df.Puntos.astype('int')

#df.head()

In [ ]:
df.shape

In [ ]:
df.Puntos.fillna(0,inplace=True)
df.isna().sum().sort_values(ascending=False)

In [ ]:
df.dropna(how="any",inplace=True)

In [ ]:
pd.DataFrame(df[df.Puntos < 0])

In [ ]:
df.drop(columns=['CustomerId'],inplace=True)

In [ ]:
clusters = [-np.inf,-1,1799,4599,5999,6499,7999,np.inf]
nombres_clusters =["negativos","0 a 1799","1800 a 4599","4600 a 5999","6000 a 6499","6500 a 7999","mas de 8000"]
df['segmento_puntos'] = pd.cut(df.Puntos,bins= clusters, labels = nombres_clusters)

In [ ]:
for grupo in ["1800 a 4599","4600 a 5999","6000 a 6499","6500 a 7999","mas de 8000"]:
    max_MB=78
    df_grupo = df[df["segmento_puntos"] == grupo]
    df_grupo.drop('segmento_puntos',axis=1,inplace=True)
    path = r'C:/Users/pnovero/Documents/Extractos DW/' + grupo + '.csv'
    df_grupo.to_csv(path,index=False)
    tamaño = os.path.getsize(path)/1000000
    print(tamaño)
    if tamaño > max_MB:
        os.remove(path)
        cantidad_archivos= math.ceil(tamaño/max_MB)
        archivos= np.array_split(df_grupo,cantidad_archivos)
        for idx,archivo in enumerate(archivos):
            path = r'C:/Users/pnovero/Documents/Extractos DW/' + grupo + '_' +str(idx) + '.csv'
            archivo.to_csv(path,index=False)
            print(path)